In [ ]:
import cv2
import glob
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import subprocess
import csv

In [ ]:
day_files_path = '/media/colin/box_data/ir_data/nuance_data/columbus_garrage_day_night/garrage_roof_nuance_jul17_5pm_1/cam_3/matlab_clahe2'
day_descriptors_path = '/media/colin/box_data/ir_data/nuance_data/columbus_garrage_day_night/garrage_roof_nuance_jul17_5pm_1/cam_3/matlab_clahe2/irpoint_v5/descriptors'
night_files_path = '/media/colin/box_data/ir_data/nuance_data/columbus_garrage_day_night/garrage_roof_nuance_jul14_7pm/cam_3/matlab_clahe2'
night_descriptors_path = '/media/colin/box_data/ir_data/nuance_data/columbus_garrage_day_night/garrage_roof_nuance_jul14_7pm/cam_3/matlab_clahe2/irpoint_v5/descriptors'
bow_executable = '/home/colin/Research/ir/DBoW2/build/simple_ir_test'
temp_dir = '/home/colin/Research/ir/DBoW2/tmp'
# vocab_file = '/media/colin/box_data/ir_data/dbow2_vocabularies/irpoint_k4_L10.yml.gz'
vocab_file = '/media/colin/box_data/ir_data/dbow2_vocabularies/irpoint_k5_L10.yml.gz'
output_file = temp_dir + "/result"

day_image_files = sorted(glob.glob(os.path.join(day_files_path,"*.png")))
night_image_files = sorted(glob.glob(os.path.join(night_files_path,"*.png")))
day_descriptor_files = sorted(glob.glob(os.path.join(day_descriptors_path,"*.npy")))
night_descriptor_files = sorted(glob.glob(os.path.join(night_descriptors_path,"*.npy")))
n_images = 100
n_skip = int(len(day_image_files)/n_images)+1

# offset = 10
night_image_files = [f for i, f in enumerate(night_image_files) if i%n_skip==0]
day_image_files = [f for i, f in enumerate(day_image_files) if i%n_skip==0]
print("selected {} images".format(len(day_image_files)))
day_image_basenames = [os.path.basename(p)[:-4] for p in day_image_files]
night_image_basenames = [os.path.basename(p)[:-4] for p in night_image_files]
day_descriptor_files = [os.path.join(day_descriptors_path, b+".npy") for b in day_image_basenames]
night_descriptor_files = [os.path.join(night_descriptors_path, b+".npy") for b in night_image_basenames]
# _ = [print( os.path.isfile(p)) for p in descriptor_files]

#write descriptor file names to temp file
desc_txt_path = os.path.join(temp_dir,"dirs.txt")
with open(desc_txt_path,'w') as file:
    for d_path in day_descriptor_files[:-1]:
        file.write(d_path + '\n')
    file.write(day_descriptor_files[-1])


#read images and descriptors
querry_id = 20
# querry_id = 90
querry_image_path = night_image_files[querry_id]
querry_image = cv2.imread( querry_image_path, cv2.IMREAD_GRAYSCALE)
images = [cv2.imread(i, cv2.IMREAD_GRAYSCALE) for i in day_image_files]
descriptors = [np.load(d) for d in day_descriptor_files]

# #show images
# fig, (ax1, ax2) = plt.subplots(1,2)
# _ = ax1.imshow(querry_image)
# ax1.set_title('querry_image')
# _ = ax2.imshow(images[querry_id])
# ax2.set_title('closest_image')

# print(os.path.basename(offset_image_files[querry_id])[:-4])
# print(os.path.basename((image_files[querry_id]))[:-4])

In [ ]:
cmd = [bow_executable, night_descriptor_files[querry_id], desc_txt_path, vocab_file, output_file]
# print(" ".join(cmd))

process = subprocess.Popen(cmd)
process.wait()

In [ ]:
#read results
results = []
with open(output_file, 'r') as csv_file:
    reader = csv.reader(csv_file, delimiter = ',')
    for row in reader:
        results.append([int(row[0]), float(row[1])])


In [ ]:
#show results
fig, axes = plt.subplots(1,len(results)+1, figsize = (20,60))
axes[0].imshow(querry_image)
axes[0].set_title("Querry Image")
axes[0].axis('off')
for i, res in enumerate(results):
    idx = res[0]
    score = res[1]
    result_image = images[idx]
    i = i +1
    axes[i].imshow(result_image)
    axes[i].set_title("score = {}".format(score))
    axes[i].axis('off')